# Lead Harvesting Notebook LM   AI-powered pipeline for scraping public records (probate, evictions, code violations, tax delinquencies), enriching with LLMs, and routing to Google Sheets + Podio CRM, with optional REIRail skip-tracing.
> **Author:** Generated by ChatGPT on 2025-06-07 08:52 UTC


In [ ]:

# ╔══════════════════════════════════════════════╗
# ║        ENVIRONMENT & API CREDENTIALS         ║
# ╚══════════════════════════════════════════════╝
import os

# --- OpenAI / DeepSeek ----
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')  # or DEEPSEEK_API_KEY

# --- Google Service Account ----
GOOGLE_CREDS_JSON = os.getenv('GOOGLE_CREDS_JSON_PATH', 'google_creds.json')
SPREADSHEET_NAME = os.getenv('SPREADSHEET_NAME', 'LeadHarvestingNotebook')

# --- Podio ----
PODIO_TOKEN = os.getenv('PODIO_TOKEN')
PODIO_APP_ID = os.getenv('PODIO_APP_ID')

# --- REIRail ----
REIRAIL_API_KEY = os.getenv('REIRAIL_API_KEY')


In [ ]:

# ╔══════════════════════════════════════════════╗
# ║                 LIBRARIES                    ║
# ╚══════════════════════════════════════════════╝
import requests, json, time, re
from bs4 import BeautifulSoup
import pandas as pd

# Google Sheets
import gspread
from oauth2client.service_account import ServiceAccountCredentials


In [ ]:

# ╔══════════════════════════════════════════════╗
# ║           SCRAPER TEMPLATE FUNCTION          ║
# ╚══════════════════════════════════════════════╝
def scrape_code_violations(url: str) -> pd.DataFrame:
    '''Scrape code-violation table from city portal to DataFrame'''
    headers = {
        'User-Agent': 'Mozilla/5.0 (LeadHarvestingBot/0.1)'
    }
    r = requests.get(url, headers=headers, timeout=30)
    soup = BeautifulSoup(r.text, 'html.parser')

    rows = []
    for tr in soup.select('table tr')[1:]:  # skip header
        tds = [td.get_text(strip=True) for td in tr.find_all('td')]
        if len(tds) >= 4:
            rows.append({
                'address': tds[0],
                'violation': tds[1],
                'date_reported': tds[2],
                'status': tds[3]
            })
    df = pd.DataFrame(rows)
    return df


In [ ]:

# ╔══════════════════════════════════════════════╗
# ║        LLM ENRICHMENT & MOTIVATION SCORE     ║
# ╚══════════════════════════════════════════════╝
import openai

def score_motivation_llm(row: dict, model: str = 'gpt-4o-mini') -> int:
    '''Return 1-5 motivation score using OpenAI or DeepSeek'''
    prompt = f"Rate seller motivation 1-5 (5 = desperate, 1 = unlikely) given: Address: {row['address']} Violation: {row['violation']} Status: {row['status']} Reported: {row['date_reported']} Return only the integer."
    try:
        resp = openai.ChatCompletion.create(
            api_key=OPENAI_API_KEY,
            model=model,
            messages=[{'role': 'user', 'content': prompt}],
            temperature=0
        )
        score = int(re.findall(r'\d', resp.choices[0].message.content)[0])
    except Exception as e:
        print('LLM error', e)
        score = 0
    return score


In [ ]:

# ╔══════════════════════════════════════════════╗
# ║         GOOGLE SHEETS INTEGRATION            ║
# ╚══════════════════════════════════════════════╝
def get_sheet(sheet_name=SPREADSHEET_NAME):
    scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
    creds = ServiceAccountCredentials.from_json_keyfile_name(GOOGLE_CREDS_JSON, scope)
    client = gspread.authorize(creds)
    return client.open(sheet_name).sheet1  # first tab

def append_to_sheet(df: pd.DataFrame):
    sh = get_sheet()
    rows = df.values.tolist()
    sh.append_rows(rows, value_input_option='RAW')


In [ ]:

# ╔══════════════════════════════════════════════╗
# ║               PODIO PUSH                     ║
# ╚══════════════════════════════════════════════╝
def push_to_podio(item: dict):
    url = f'https://api.podio.com/item/{PODIO_APP_ID}'
    headers = {'Authorization': f'OAuth2 {PODIO_TOKEN}', 'Content-Type': 'application/json'}
    data = {
        'fields': {
            'address': item['address'],
            'violation': item['violation'],
            'status': item['status'],
            'date_reported': item['date_reported'],
            'motivation_score': item['motivation_score']
        }
    }
    r = requests.post(url, headers=headers, data=json.dumps(data))
    if r.status_code != 200:
        print('Podio error', r.text)


In [ ]:

# ╔══════════════════════════════════════════════╗
# ║           OPTIONAL REIRAIL ENRICHMENT        ║
# ╚══════════════════════════════════════════════╝
def reirail_skip_trace(address: str) -> dict:
    '''Stub for REIRail API - replace with actual endpoint'''
    url = 'https://api.reirail.com/skiptrace'
    payload = {'address': address, 'token': REIRAIL_API_KEY}
    try:
        r = requests.post(url, json=payload, timeout=15)
        return r.json()
    except Exception as e:
        print('REIRail error', e)
        return {}


In [ ]:

# ╔══════════════════════════════════════════════╗
# ║                  PIPELINE                    ║
# ╚══════════════════════════════════════════════╝
def run_pipeline(source_url: str):
    df = scrape_code_violations(source_url)
    if df.empty:
        print('No rows scraped')
        return

    # Score with LLM
    df['motivation_score'] = df.apply(lambda r: score_motivation_llm(r), axis=1)

    # Append to Google Sheets
    append_to_sheet(df)

    # Push to Podio & REIRail
    for _, row in df.iterrows():
        item = row.to_dict()
        push_to_podio(item)
        # Optionally enrich with REIRail
        skip = reirail_skip_trace(item['address'])
        if skip:
            print('Skip-trace result:', skip)
